# 4. Train and predict

So far, we learn how to make PyTorch tensor, modules and model. <br>
Now, let's dive into training, optimizing and making predictions.

Basic workflow is as follows:
- Load and preprocess data
- Define model
- Build model
- Train model
- Predict on test data

## Import

In [ ]:
import numpy as np
import torch
import torch.nn as nn

np.random.seed(1234)
torch.manual_seed(1234)

if torch.cuda.is_available(): 
    torch.cuda.manual_seed(1234)
    torch.cuda.manual_seed_all(1234)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

## CPU or GPU

Before you load data or build model, you have to decide which device your codes will be running between CPU and GPU. <br>
PyTorch provides a function to check if you can use GPU. (```torch.cuda.is_available()```) <br>
Once you choose the device, using ```.to(device)```, send your data (torch tensor), modules (including your model) to the device.

In [ ]:
cuda_available = torch.cuda.is_available()
print(f'Is cuda available? : {cuda_available}')

device = 'cuda' if cuda_available else 'cpu'
print(f'Device to use: {device}')

Is cuda available? : True
Device to use: cuda


## Load data

In [ ]:
# 5 Random MNIST shape data with random label
x = torch.rand(5, 1, 28, 28).to(device)
y = torch.LongTensor([0, 3, 1, 2, 4]).to(device)

## Define model
Make your own model! Review the third tutorial **3_Building_model.ipynb**.

In [ ]:
class MyClassifier(nn.Module):
    def __init__(self):
        super(MyClassifier, self).__init__()
        self.conv_1 = nn.Conv2d(1, 3, 1, 1)
        self.conv_2 = nn.Conv2d(3, 3, 1, 1)
        self.maxpool = nn.MaxPool2d(2, 2)
        self.fc_layer = nn.Linear(588, 5)
        self.softmax = nn.Softmax(dim=-1)
    
    def forward(self, x):
        num_data = x.shape[0]
        out = self.conv_1(x)
        out = self.conv_2(out)
        out = self.maxpool(out)
        out = self.fc_layer(out.view(num_data, -1))
        # return self.softmax(out)
        return out

## Build model
Once you define model, build your model and send it to the device!

In [ ]:
model = MyClassifier().to(device)

## Train model

Training is very similar with what we did in past assignments; define loss function, optimizer, optimize model for epochs. <br>
For simplicity, we do not consider batch training here.

**Loss function** <br>
For multiclass classification, we have been using cross-entropy loss. <br>
In PyTorch, it is also a module, called ```nn.CrossEntropyLoss()```. <br>
BE CAREFUL: ```nn.CrossEntropyLoss()``` is **softmax + negative log loss**, which means you have to pass score BEFORE softmax. <br>
If you want to use softmax function, then ```nn.NLLLoss()``` is the right choice. <br>
For more loss functions, see official documents [Loss functions](https://pytorch.org/docs/stable/nn.html#loss-functions)

In [ ]:
learning_rate = 0.01

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
# Clear gradient to zero!
optimizer.zero_grad()

# Model output
out = model(x)

# Calculate loss
loss = criterion(out, y)
print(f'loss = {loss}')

loss = 1.634621024131775


In [ ]:
# Compute gradient
loss.backward()

# Update weight
optimizer.step()

## PyTorch Tensor to Numpy

In [ ]:
print(out.device)

out_cpu = out.detach().cpu()
print(out_cpu.device)

out_numpy = out_cpu.numpy()
print(type(out_numpy))

cuda:0
cpu
<class 'numpy.ndarray'>


In [ ]:
pred = out_numpy.argmax(-1)
print(f'output: {out_numpy[0]}')
print(f'prediction: {pred[0]}')

output: [ 0.02061159 -0.06234899 -0.3941541   0.15722708  0.03442803]
prediction: 3


# Conclusion
So far, we have briefly looked at essence of PyTorch. <br>
Hoping that it helped you start your project, we provide several great PyTorch tutorials to check. <br>
Good luck! <br>

- [PyTorch Official Tutorial](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html)
- [Tutorial by Yunjey Choi](https://github.com/yunjey/pytorch-tutorial)
- [Tutorial by morvanzhou](https://github.com/MorvanZhou/PyTorch-Tutorial)
- [모두를 위한 딥러닝 시즌2](https://github.com/deeplearningzerotoall/PyTorch)